<img src="https://github.com/djp840/MSDS_422_Public/blob/master/images/NorthwesternHeader.png?raw=1">

## MSDS422 Assignment 02:

<div class="alert alert-block alert-success">
    <b>More Technical</b>: Throughout the notebook. This types of boxes provide more technical details and extra references about what you are seeing. They contain helpful tips, but you can safely skip them the first time you run through the code.
</div>

### Data Dictionary Housing Values in Suburbs of Boston

The Boston data frame has 506 rows and 14 columns.<br>
The <b>medv variable</b> is the target variable.<br>
<br>
<b>crim</b><br>
per capita crime rate by town.<br>
<br>
<b>zn</b><br>
proportion of residential land zoned for lots over 25,000 sq.ft.<br>
<br>
<b>inducrims</b><br>
proportion of non-retail business acres per town.<br>
<br>
<b>chas</b><br>
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).<br>
<br>
<b>nox</b><br>
nitrogen oxides concentration (parts per 10 million).<br>
<br>
<b>rm</b><br>
average number of rooms per dwelling.<br>
<br>
<b>age</b><br>
proportion of owner-occupied units built prior to 1940.<br>
<br>
<b>dis</b><br>
weighted mean of distances to five Boston employment centres.<br>
<br>
<b>rad</b><br>
index of accessibility to radial highways.<br>
<br>
<b>tax</b><br>
full-value property minus tax rate per ten thousand dollars<br>
<br>
<b>ptratio</b><br>
pupil-teacher ratio by town.<br>
<br>
<b>black</b><br>
1 Thousand(Bk - 0.63)^2" where Bk is the proportion of blacks by town.<br>
<br>
<b>lstat</b><br>
lower status of the population (percent).<br>
<br>
<b>medv</b><br>
median value of owner-occupied homes in $1000s.<br>
<br>
<br>
<b>Sources:</b><br>
Harrison, D. and Rubinfeld, D.L. 1978 Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.<br>
<br>
Belsley D.A., Kuh, E. and Welsch, R.E. 1980 Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.<br>

## Import packages 

In [ ]:
import numpy as np
import pandas as pd
import os
import itertools
from math import sqrt
from scipy import stats as st
#import cvxopt

import sklearn 
from sklearn.preprocessing import StandardScaler # used for variable scaling data
from sklearn.preprocessing import MinMaxScaler as Scaler # used for variable scaling data
from sklearn.model_selection import train_test_split

import sklearn.linear_model 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor # Random Forest package
from sklearn.ensemble import ExtraTreesRegressor # Extra Trees package
from sklearn.ensemble import GradientBoostingRegressor # Gradient Boosting package

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.metrics import make_scorer, accuracy_score 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

from matplotlib import pyplot as plt
from matplotlib import rc
import seaborn as sns
sns.set_style("whitegrid")
sns.set(style="whitegrid", color_codes=True)
plt.rc("font", size=14)

In [ ]:
%matplotlib inline

<div class="alert alert-block alert-info">
    <b>Suppress warning messages</b></div>

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

### Load Data (Local Directory)

In [ ]:
boston_df=pd.read_csv('./data/MSDS422_02_boston.csv')

### Mount Google Drive to Colab Enviorment

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

### Data Quality Review 

In [ ]:
print("Shape:", boston_df.shape,"\n")
print("Variable Types:") 
print(boston_df.dtypes)

In [ ]:
boston_df.head()

 <div class="alert alert-block alert-warning">
Dropping Neighborhood as it is non-numeric
 </div>

In [ ]:
boston_df=boston_df.drop('neighborhood', 1)

## Exploritory Data Analysis (EDA) 

### Review Dataset for Missing Values

In [ ]:
boston_df.isnull().sum()

<div class="alert alert-block alert-warning">
Attention to the <b>count</b> row for each column number of records
</div> 

In [ ]:
boston_df.describe()

### Review Dataset Distributions  Boxplot and Histograms

In [ ]:
boston_df.boxplot(vert=False, figsize=(10,10), grid=False)

In [ ]:
sns.pairplot(boston_df, diag_kind='hist')

## Preprocess Data for Analysis

#### Normalizing Variable Distributions with Log Transformation 

Analysis of linear relationships between variables can introduce "0" (zero) values, these have to be removed to be able to work with Log transformation (normalization) of the data

<div class="alert alert-block alert-info">
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html
    </div>

y = (x**lmbda - 1) / lmbda,  for lmbda > 0<br>
    log(x),                  for lmbda = 0<br>

<b>boxcox</b> requires the input data to be positive. Sometimes a Box-Cox transformation provides a `shift parameter` to achieve this;<br> <b>boxcox</b> does not. Such a shift parameter is equivalent to adding a positive constant to x before calling boxcox.

#### Create Dataframe with Target Variable

In [ ]:
boston_df1=boston_df.copy()

In [ ]:
columns = ['crim','zn','indus','chas','nox','rooms','age','dis','rad','tax','ptratio','lstat']
boston_Target = boston_df1.drop(columns=columns)

In [ ]:
print(boston_Target.head())

In [ ]:
boston_Target.shape

In [ ]:
boston_df2=boston_df.apply(lambda x: x+.01)
boston_df2=boston_df2.transform(lambda x: st.boxcox(x)[0])

In [ ]:
sns.pairplot(boston_df2, diag_kind='hist')

In [ ]:
boston_df3=boston_df2.transform(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
boston_df3.hist(figsize=(10,10))

#### Plot the distribution of the target variable mv (median value of owner-occupied homes in $1000s)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.distplot(boston_Target['mv'], bins=30)
plt.show()

In [ ]:
boston_df3.dtypes

The <b>medv variable</b> is the target variable.<br>

In [ ]:
cols = boston_df3.columns.tolist()
cols = cols[-1:] + cols[:-1]
boston_df4=boston_df3[cols]
boston_df4.describe(include="all")

<div class="alert alert-block alert-info">
    <b>Correlation matrix that measures the linear relationships</b><br> 
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html
    </div>

In [ ]:
plt.figure(figsize=(15,10))
corr=boston_df4.corr(method='pearson')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask=mask, cmap=sns.diverging_palette(220, 10, as_cmap=True), annot=True, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

#### Create Dataset for Random Search

In [ ]:
boston_df6=boston_df4.copy()

#### Drop Correlated Values ( correlation >= (+/-) 0.75 )

In [ ]:
columns = ['rad', 'nox','dis']
boston_df5 = boston_df4.drop(columns=columns)

In [ ]:
list(boston_df5.columns.values)

In [ ]:
# Create multiple plots
features = boston_df5.drop('mv', 1).columns
target = boston_df5['mv']
plt.figure(figsize=(20,20))
for index, feature_name in enumerate(features):
    plt.subplot(4,len(features)/2, index+1)
    plt.scatter(boston_df5[feature_name], target)
    plt.title(feature_name, fontsize=15)
    plt.xlabel(feature_name, fontsize=8)
    plt.ylabel('mv', fontsize=15)

The variables room and lstat look to have linear relationship with mv target variable

## Create Linear Regression Model

### Summary Statistics for Linear Regression Model - Statsmodel 

In [ ]:
X = pd.DataFrame(np.c_[boston_df['crim']
,boston_df['indus']
,boston_df['rooms']
,boston_df['age']
,boston_df['tax']
,boston_df['ptratio']
,boston_df['lstat']])

In [ ]:
Y = boston_Target['mv']

In [ ]:
X=sm.add_constant(X)
model=sm.OLS(Y, X)

In [ ]:
results=model.fit()

In [ ]:
print(results.summary())

<div class="alert alert-block alert-info">
<b>sklearn.linear_model.LinearRegression</b><br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
</div>

In [ ]:
X = boston_df5[[ 'crim'
                 ,'indus'
                 ,'rooms'
                 ,'age'
                 ,'tax'
                 ,'ptratio'
                 ,'lstat']]
y = boston_Target.mv

#### Split Dataset into Training and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 10)

#### Review Train and Testing 

In [ ]:
print(X_train.shape) 
print(X_test.shape) 
print(y_train.shape) 
print(y_test.shape) 

#### Create LinearRegression Instance

In [ ]:
lrm = LinearRegression()

# Fit data on to the model
lrm.fit(X_train, y_train)

# Predict
y_predicted_lrm = lrm.predict(X_test)

#### Linear Regression Model Actual Vs. Predicted Price Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(y_test, y_predicted_lrm)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.ylabel('Predicted Prices', fontsize=20);
plt.xlabel('Actual Prices', fontsize=20);
plt.title("Linear Regression Predicted Boston Housing Prices vs. Actual in $1000's", fontsize=20)

plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

plt.show()

In [ ]:
print("Linear Regression R_squared = ",lrm.score(X,y)) 
pred= lrm.predict(X)
rmse = sqrt(mean_squared_error(pred, y))
print('Linear Regression RMSE = ', rmse)

In [ ]:
print(lrm.coef_)

In [ ]:
print(lrm.intercept_)

## Create Ridge Regression Model

<div class="alert alert-block alert-success">
    <b>Ridge Regression</b>: tends to give small but well distributed weights, because the l2 regularization cares more about driving big weight to small weights, instead of driving small weights to zeros. If you only have a few predictors, and you are confident that all of them should be really relevant for predictions, try Ridge as a good regularized linear regression method
</div>

<div class="alert alert-block alert-info">
<b>sklearn.linear_model.Ridge</b><br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
</div>

#### Create Ridge Regression Instance

In [ ]:
rrm = Ridge()

# Fit data on to the model
rrm.fit(X_train, y_train)

# Predict
y_predicted_rrm = rrm.predict(X_test)

#### Linear Regression Model Actual Vs. Predicted Price Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(y_test, y_predicted_rrm)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.ylabel('Predicted Prices', fontsize=20);
plt.xlabel('Actual Prices', fontsize=20);
plt.title("Ridge Regression Predicted Boston Housing Prices vs. Actual in $1000's", fontsize=20)

plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

plt.show()

In [ ]:
print("Ridge Regression R_squared = ",rrm.score(X,y)) 
pred= rrm.predict(X)
rmse = sqrt(mean_squared_error(pred, y))
print('Ridge Regression RMSE = ', rmse)

In [ ]:
print(rrm.coef_)

In [ ]:
print(rrm.intercept_)

## Create Lasso Regression Model
Linear Model trained with L1 prior as regularizer (Lasso)

<div class="alert alert-block alert-success">
    <b>Lasso Regression</b>: tend to give sparse weights (most zeros), because the l1 regularization cares equally about driving down big weights to small weights, or driving small weights to zeros. If you have a lot of predictors (features), and you suspect that not all of them are that important, Lasso and ElasticNet may be really good idea to start with</div>

<div class="alert alert-block alert-info">
<b>sklearn.linear_model.Lasso</b><br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
</div>

In [ ]:
larm = Lasso(alpha=0.001)

# Fit data on to the model
larm.fit(X_train, y_train)

# Predict
y_predicted_larm = larm.predict(X_test)

#### Lasso Regression Model Actual Vs. Predicted Price Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(y_test,y_predicted_larm)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.ylabel('Predicted Prices', fontsize=20);
plt.xlabel('Actual Prices', fontsize=20);
plt.title("Lasso Regression Predicted Boston Housing Prices vs. Actual in $1000's", fontsize=20)

plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

plt.show()

In [ ]:
print("Lasso Regression R_squared = ",larm.score(X,y)) 
pred= larm.predict(X)
rmse = sqrt(mean_squared_error(pred, y))
print('Lasso Regression RMSE = ', rmse)

In [ ]:
print(larm.coef_)

In [ ]:
print(larm.intercept_)

## Create ElasticNet Regression Model

<div class="alert alert-block alert-success">
    <b>ElasticNet Regression</b>: tend to give sparse weights (most zeros), because the l1 regularization cares equally about driving down big weights to small weights, or driving small weights to zeros. If you have a lot of predictors (features), and you suspect that not all of them are that important, Lasso and ElasticNet may be really good idea to start with</div>

<div class="alert alert-block alert-info">
<b>sklearn.linear_model.ElasticNet</b><br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
</div>

#### Create ElasticNet Regression Instance

In [ ]:
enrm = ElasticNet(alpha=0.001)

# Fit data on to the model
enrm.fit(X_train, y_train)

# Predict
y_predicted_enrm = enrm.predict(X_test)

#### ElasticNet Regression Model Actual Vs. Predicted Price Plot

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(y_test, y_predicted_enrm)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.ylabel('Predicted Prices', fontsize=20);
plt.xlabel('Actual Prices', fontsize=20);
plt.title("ElasticNet Regression Predicted Boston Housing Prices vs. Actual in $1000's", fontsize=20)

plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

plt.show()

In [ ]:
print("ElasticNet Regression R_squared = ",enrm.score(X,y)) 
pred= enrm.predict(X)
rmse = sqrt(mean_squared_error(pred, y))
print('ElasticNet Regression RMSE = ', rmse)

In [ ]:
print(enrm.coef_)

In [ ]:
print(enrm.intercept_)

#### Create copy for Model Development 

In [ ]:
model_data=boston_df6.values

#### Models (Linear Regression, Ridge Regression, Lasso Regression, Elastic Net Regression)

In [ ]:
# Seed value for random number generators to obtain reproducible results
RANDOM_SEED = 1

# The model input data outside of the modeling method calls
names = ['Linear_Regression', 'Ridge_Regression', 'Lasso_Regression', 'ElasticNet_Regression']

# Specify the set of regression models being evaluated (we set normalize=False because we have standardized above)
regressors = [LinearRegression(fit_intercept = True, normalize = False), 
              Ridge(alpha = 75, solver = 'cholesky', fit_intercept = True, normalize = False, random_state = RANDOM_SEED),
              Lasso(alpha = 0.01, max_iter=10000, tol=0.01, fit_intercept = True, normalize = False, random_state = RANDOM_SEED),
              ElasticNet(alpha = 0.01, l1_ratio = 0.5, max_iter=10000, tol=0.01, fit_intercept = True, normalize = False, random_state = RANDOM_SEED),
              ]

### Random Search

In [ ]:
# Establish number of cross folds employed for cross-validation
N_FOLDS = 10

# Setup numpy array for storing results
cv_results = np.zeros((N_FOLDS, len(names)))

# Initiate splitting process
kf = KFold(n_splits = N_FOLDS, shuffle=False, random_state = RANDOM_SEED)

# Check the splitting process by looking at fold observation counts
index_for_fold = 0  # Fold count initialized 
for train_index, test_index in kf.split(model_data):
    print('\nFold index:', index_for_fold, '---------------------------------------------------------------------------------------')

# The structure of modeling data for this study has the response variable coming first and explanatory variables later          
# so 1:model_data.shape[1] slices for explanatory variables and 0 is the index for the response variable    
    X_train = model_data[train_index, 1:model_data.shape[1]]
    X_test = model_data[test_index, 1:model_data.shape[1]]
    y_train = model_data[train_index, 0]
    y_test = model_data[test_index, 0]   

    index_for_method = 0  # Method count initialized
    for name, reg_model in zip(names, regressors):
        reg_model.fit(X_train, y_train)  # Fit on the train set for this fold
 
        # Evaluate on the test set for this fold
        y_test_predict = reg_model.predict(X_test)
        fold_method_result = sqrt(mean_squared_error(y_test, y_test_predict))
        cv_results[index_for_fold, index_for_method] = fold_method_result
        index_for_method += 1
  
    index_for_fold += 1

cv_results_df = pd.DataFrame(cv_results)
cv_results_df.columns = names

print('\n---------------------------------------------------------------------------------------')
print('Average results from ', N_FOLDS, '-fold cross-validation\n',
      'in standardized units (mean 0, standard deviation 1)\n',
      '\nMethod               Root mean-squared error', sep = '')    
print(cv_results_df.mean())

In [ ]:
cv_results_df.head(10)